In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import h5py

In [8]:
myFile = h5py.File('../Input/images_training.h5', 'r')
# The '...' means retrieve the whole tensor
X_train = myFile['data'][...]
myLabel = h5py.File('../Input/labels_training.h5', 'r')
y_train = myLabel['label'][...]
X_train = X_train.reshape(len(X_train), -1)
print(X_train.shape)
print(y_train.shape)

(30000, 784)
(30000,)


In [14]:
Test = h5py.File('../Input/images_testing.h5', 'r')
X_test = Test['data'][...]
testLabel = h5py.File('../Input/labels_testing_2000.h5', 'r')
y_test = testLabel['label'][...]
X_test = X_test[:2000,:]
X_test = X_test.reshape(len(X_test), -1)
print(X_test.shape)
print(y_test.shape)

(2000, 784)
(2000,)


In [23]:
torch_X_train = torch.from_numpy(X_train)
torch_y_train = torch.from_numpy(y_train)
torch_X_test = torch.from_numpy(X_test)
torch_y_test = torch.from_numpy(y_test)
print('shape of X_train', torch_X_train.shape)
print('shape of y_train', torch_y_train.shape)
print('shape of X_test', torch_X_test.shape)
print('shape of y_test', torch_y_test.shape)

shape of X_train torch.Size([30000, 784])
shape of y_train torch.Size([30000])
shape of X_test torch.Size([2000, 784])
shape of y_test torch.Size([2000])


In [27]:
trainloader = torch.utils.data.DataLoader(X_train, batch_size=8,\
                                         shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(X_test, batch_size=8,\
                                         shuffle=False, num_workers=2)
classes = ('T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot')
print('length of trainloader:', len(trainloader))
print('length of testloader:', len(testloader))

length of trainloader: 3750
length of testloader: 250


In [59]:
import torch
import torchvision
import torchvision.transforms as transforms


transform = transforms.ToTensor()

trainset = torchvision.datasets.FashionMNIST(root='./Fashion MNIST', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='./Fashion MNIST', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False, num_workers=2)

classes = ('T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot')

In [60]:
print('length of trainloader:', len(trainloader))
print('length of testloader:', len(testloader))

length of trainloader: 3750
length of testloader: 625


In [84]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.BN1 = nn.BatchNorm2d(32, eps=1e-5, momentum=0.1, affine=True)
        self.BN2 = nn.BatchNorm2d(64, eps=1e-5, momentum=0.1, affine=True)
        self.fc1 = nn.Linear(64 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.BN1(self.conv1(x))))
        x = self.pool(F.relu(self.BN2(self.conv2(x))))
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        x = F.log_softmax(self.fc3(x), dim=1)
        return x


net = Net()

In [85]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [86]:
%%time
average_loss_series = []

for epoch in range(16):  # loop over the dataset multiple times
    
    running_loss = 0.0

    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 750 == 749:    # print every 2000 mini-batches
            average_loss = running_loss/750
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, average_loss))
            average_loss_series.append(average_loss)
            running_loss = 0.0
print('Finished Training')

[1,   750] loss: 1.071
[1,  1500] loss: 0.508
[1,  2250] loss: 0.448
[1,  3000] loss: 0.396
[1,  3750] loss: 0.368
[2,   750] loss: 0.344
[2,  1500] loss: 0.343
[2,  2250] loss: 0.331
[2,  3000] loss: 0.317
[2,  3750] loss: 0.312
[3,   750] loss: 0.289
[3,  1500] loss: 0.292
[3,  2250] loss: 0.294
[3,  3000] loss: 0.276
[3,  3750] loss: 0.279
[4,   750] loss: 0.260
[4,  1500] loss: 0.257
[4,  2250] loss: 0.243
[4,  3000] loss: 0.253
[4,  3750] loss: 0.258
[5,   750] loss: 0.234
[5,  1500] loss: 0.230
[5,  2250] loss: 0.233
[5,  3000] loss: 0.237
[5,  3750] loss: 0.235
[6,   750] loss: 0.203
[6,  1500] loss: 0.206
[6,  2250] loss: 0.221
[6,  3000] loss: 0.221
[6,  3750] loss: 0.226
[7,   750] loss: 0.201
[7,  1500] loss: 0.197
[7,  2250] loss: 0.194
[7,  3000] loss: 0.206
[7,  3750] loss: 0.208
[8,   750] loss: 0.174
[8,  1500] loss: 0.182
[8,  2250] loss: 0.182
[8,  3000] loss: 0.194
[8,  3750] loss: 0.194
[9,   750] loss: 0.164
[9,  1500] loss: 0.168
[9,  2250] loss: 0.173
[9,  3000] 

In [87]:
%%time
test_correct = []
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(testloader):
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if i % 250 == 249:
            test_correct.append(correct / total)
print('Accuracy of the network on the 10000 test images: {:.1f}%'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 90.4%
Wall time: 6.8 s


## Another better CNN with BN and softmax

In [100]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=1, stride=2)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.AvgPool2d(kernel_size=1, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        #self.pool = nn.MaxPool2d(2, 2)
        self.BN1 = nn.BatchNorm2d(32, eps=1e-5, momentum=0.1, affine=True)
        self.BN2 = nn.BatchNorm2d(64, eps=1e-5, momentum=0.1, affine=True)
        self.fc1 = nn.Linear(64 * 4 * 4, 64)#pool3 要否
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.BN1(self.conv1(x))))
        x = self.pool2(F.relu(self.BN1(self.conv2(x))))
        x = self.pool2(F.relu(self.BN2(self.conv3(x))))
        #print(x.shape)
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        x = F.log_softmax(self.fc3(x), dim=1)
        return x


net = Net()

In [101]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [102]:
%%time
average_loss_series = []

for epoch in range(16):  # loop over the dataset multiple times
    
    running_loss = 0.0

    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 750 == 749:    # print every 1125 mini-batches
            average_loss = running_loss/750
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, average_loss))
            average_loss_series.append(average_loss)
            running_loss = 0.0
print('Finished Training')

[1,   750] loss: 0.690
[1,  1500] loss: 0.444
[1,  2250] loss: 0.397
[1,  3000] loss: 0.365
[1,  3750] loss: 0.348
[2,   750] loss: 0.313
[2,  1500] loss: 0.305
[2,  2250] loss: 0.297
[2,  3000] loss: 0.301
[2,  3750] loss: 0.289
[3,   750] loss: 0.260
[3,  1500] loss: 0.269
[3,  2250] loss: 0.255
[3,  3000] loss: 0.260
[3,  3750] loss: 0.254
[4,   750] loss: 0.219
[4,  1500] loss: 0.240
[4,  2250] loss: 0.236
[4,  3000] loss: 0.234
[4,  3750] loss: 0.225
[5,   750] loss: 0.202
[5,  1500] loss: 0.213
[5,  2250] loss: 0.213
[5,  3000] loss: 0.210
[5,  3750] loss: 0.217
[6,   750] loss: 0.184
[6,  1500] loss: 0.189
[6,  2250] loss: 0.200
[6,  3000] loss: 0.200
[6,  3750] loss: 0.186
[7,   750] loss: 0.169
[7,  1500] loss: 0.174
[7,  2250] loss: 0.178
[7,  3000] loss: 0.177
[7,  3750] loss: 0.178
[8,   750] loss: 0.154
[8,  1500] loss: 0.149
[8,  2250] loss: 0.167
[8,  3000] loss: 0.167
[8,  3750] loss: 0.176
[9,   750] loss: 0.141
[9,  1500] loss: 0.147
[9,  2250] loss: 0.142
[9,  3000] 

In [103]:
%%time
test_correct = []
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(testloader):
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if i % 250 == 249:
            test_correct.append(correct / total)
print('Accuracy of the network on the 10000 test images: {:.1f}%'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 90.0%
Wall time: 18.2 s
